<a href="https://colab.research.google.com/github/isura3147/ObjectDetection/blob/main/FinalProto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the ultralytics package from PyPI
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 9.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="wcogxoYz6E1ZlxsgJuMh")
project = rf.workspace("evgeny-yankovsky").project("weapon-vtrsk")
version = project.version(4)
dataset = version.download("yolov8")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.1/74.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.1

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.2, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Weapon-4 in yolov8:: 100%|██████████| 23176/23176 [00:03<00:00, 7499.34it/s]


In [ ]:
import os

# Directories containing your label files
directories = {
    'train': '/content/Weapon-4/train/labels',
    'valid': '/content/Weapon-4/valid/labels',
    'test': '/content/Weapon-4/test/labels'
}


# List of class names in the order they appear in your 'names' field
class_names = ['Machinegun', 'Pistol', 'Rifle', 'Shotgun']

class_counts = {class_name: 0 for class_name in class_names}
total_entries = 0
set_counts = {set_name: 0 for set_name in directories.keys()}

for set_name, directory in directories.items():
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            with open(os.path.join(directory, filename), 'r') as file:
                for line in file:
                    class_index = int(line.split()[0])
                    class_name = class_names[class_index]
                    class_counts[class_name] += 1
                    total_entries += 1
                set_counts[set_name] += 1

print("Number of data entries: ", total_entries)
print("Number of classes: ", len(class_names))
print("Number of data entries for each class: ")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")
print("Number of images in each set: ")
for set_name, count in set_counts.items():
    print(f"{set_name}: {count}")

In [4]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Setup complete ✅ (16 CPUs, 62.8 GB RAM, 29.7/201.2 GB disk)


In [5]:
!pip install wandb -qU

# Log in to your W&B account
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/Weapon-4/data.yaml optimizer=RMSProp lr0=0.001 epochs=50 imgsz=640 batch=16

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/Weapon-4/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=RMSProp, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

In [15]:
!yolo task=detect mode=predict model=/content/runs/detect/train3/weights/best.pt conf=0.35 source=/content/Weapon-4/test/images

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients, 28.4 GFLOPs

image 1/800 /content/Weapon-4/test/images/-10-_jpg.rf.c479f799e91087546f5cbe889ada934f.jpg: 640x640 1 Machinegun, 7.3ms
image 2/800 /content/Weapon-4/test/images/-13-_jpg.rf.5b11fc36b233d6f783c2ca8390aa3e42.jpg: 640x640 1 Shotgun, 9.9ms
image 3/800 /content/Weapon-4/test/images/-13-_jpg.rf.6c8274af5f9fa142842080832e5d2668.jpg: 640x640 1 Machinegun, 9.1ms
image 4/800 /content/Weapon-4/test/images/-18-_jpg.rf.fafe0e396296a1b3d4190d3b8f83fbaf.jpg: 640x640 1 Machinegun, 9.3ms
image 5/800 /content/Weapon-4/test/images/-2-_jpg.rf.d6769043c2a05147e65ce9b660d8510e.jpg: 640x640 1 Machinegun, 8.8ms
image 6/800 /content/Weapon-4/test/images/-20-_jpg.rf.cae07f01b18bf8cc02f8af45860b94f9.jpg: 640x640 1 Machinegun, 8.8ms
image 7/800 /content/Weapon-4/test/images/-3-_jpg.rf.3c5874e124beeb9a7d99353b86b24d1b.jpg: 640x640 1 Machinegun, 8.

In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output

In [17]:
# to download the runs folder for graphs created by yolo
!zip -r sgd.zip /content/runs/detect/predict

  adding: content/runs/detect/predict/ (stored 0%)
  adding: content/runs/detect/predict/Image_58_jpg.rf.0aad7db885bd0049eafed0a610108941.jpg (deflated 4%)
  adding: content/runs/detect/predict/-8-_jpg.rf.697899cf273a1ae2f19aacbad6ebe661.jpg (deflated 11%)
  adding: content/runs/detect/predict/360_F_109680088_IThdcztHZ5lwiLrlF8drfEf4DFijJvfS_jpg.rf.4773ded01e248d7b6a642e7cd1ef084f.jpg (deflated 38%)
  adding: content/runs/detect/predict/360_F_259039184_kNVl9BH0FO2VP4dMORvDGXJOeFvrRiz2_jpg.rf.8a2b85bdc1a5d93df80ead81aec537c2.jpg (deflated 5%)
  adding: content/runs/detect/predict/pexels-photo-3015481-1-_jpg.rf.3da64e7e446c6df7597679f2e6915132.jpg (deflated 5%)
  adding: content/runs/detect/predict/images-9-_jpg.rf.7701460e286c0eb53037509af9611519.jpg (deflated 30%)
  adding: content/runs/detect/predict/pexels-photo-4836397-_jpg.rf.20382e390228e70e7d11777b9d7073f3.jpg (deflated 4%)
  adding: content/runs/detect/predict/pistol_156-_jpg.rf.4c45f4db9379b97ab181276e8dfd5182.jpg (deflated 5%)